In [9]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import csv
import pandas as pd
import os

BASE = 'https://www.schwabjobs.com'

 
def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        return dict(zip(catLinks, numPages))
        
def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(BASE + key)
    return pageLinks
        

def scrape_pages(pages):
    for p in pages:
        pageHtml = urlopen(BASE + p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = BASE + link.get('href')
            print(page)
            pageDict = scrape(page)
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('schwaData.csv'):
                    data.to_csv('schwaData.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('schwaData.csv', mode='a', header=False)
        
def scrape(page):
    
    try:
        posting = urlopen(page)
        soup2 =  BeautifulSoup(posting)
        value = soup2.findAll("div", {"class": "desc"})
    except:
        return ("An exception occurred")
    
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]
    values = [1, "Charles Schwab", 1, page, 1]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open('freq100.csv', 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]

    for val in value:
        st = val.text.split()
        for word in st:
            if(word in arr):
                ind = arr.index(word)
                wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 


                
if __name__ == '__main__':
    res = urlopen("https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
    categories = get_category(res)
    pages = get_pages(categories)
    scrape_pages(pages)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>